##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# モデルの保存と復元

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a> </td>
</table>

モデルの進行状況は、トレーニング中およびトレーニング後に保存できます。モデルが中断したところから再開できるので、長いトレーニング時間を回避できます。また、保存することによりモデルを共有したり、他の人による作業の再現が可能になります。研究モデルや手法を公開する場合、ほとんどの機械学習の実践者は次を共有します。

- モデルを構築するプログラム
- モデルのトレーニング済みモデルの重みやパラメータ

このデータを共有することで、他の人がモデルがどの様に動作するかを理解したり、新しいデータに試してみたりすることが容易になります。

注意: TensorFlow モデルはコードであり、信頼できないコードに注意する必要があります。詳細については、[TensorFlow を安全に使用する](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)をご覧ください。

### オプション

使用している API に応じて、さまざまな方法で TensorFlow モデルを保存できます。このガイドでは、高レベル API である[ tf.keras ](https://www.tensorflow.org/guide/keras)を使用して、TensorFlow でモデルを構築およびトレーニングします。他のアプローチについては、TensorFlow [保存と復元](https://www.tensorflow.org/guide/saved_model)ガイドまたは [Eager で保存する](https://www.tensorflow.org/guide/eager#object-based_saving)を参照してください。

## 設定

### インストールとインポート

TensorFlow をインストールし、依存関係インポートします。

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### サンプルデータセットの取得

ここでは、重みの保存と読み込みをデモするために、[MNIST データセット](http://yann.lecun.com/exdb/mnist/)を使います。デモの実行を速くするため、最初の 1,000 件のサンプルだけを使います。

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### モデルの定義

簡単なシーケンシャルモデルを構築することから始めます。

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

## トレーニング中にチェックポイントを保存する

再トレーニングせずにトレーニング済みモデルを使用したり、トレーニングプロセスを中断したところから再開することもできます。`tf.keras.callbacks.ModelCheckpoint` コールバックを使用すると、*トレーニング中*でも*トレーニングの終了時*でもモデルを継続的に保存できます。

### チェックポイントコールバックの使い方

トレーニング中にのみ重みを保存する `tf.keras.callbacks.ModelCheckpoint` コールバックを作成します。

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

この結果、エポックごとに更新される一連のTensorFlowチェックポイントファイルが作成されます。

In [ ]:
os.listdir(checkpoint_dir)

2 つのモデルが同じアーキテクチャを共有している限り、それらの間で重みを共有できます。したがって、重みのみからモデルを復元する場合は、元のモデルと同じアーキテクチャでモデルを作成してから、その重みを設定します。

次に、トレーニングされていない新しいモデルを再構築し、テストセットで評価します。トレーニングされていないモデルは、偶然誤差（10％ 以下の正解率）で実行されます。

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

次に、チェックポイントから重みをロードし、再び評価します。

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

### チェックポイントコールバックのオプション

このコールバックには、チェックポイントに一意な名前をつけたり、チェックポイントの頻度を調整するためのオプションがあります。

新しいモデルをトレーニングし、5 エポックごとに一意な名前のチェックポイントを保存します。

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

次に、できあがったチェックポイントを確認し、最後のものを選択します。

In [ ]:
os.listdir(checkpoint_dir)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

注意: デフォルトの TensorFlow 形式では、最新の 5 つのチェックポイントのみが保存されます。

テストのため、モデルをリセットし最後のチェックポイントを読み込みます。

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## これらのファイルは何？

上記のコードは、バイナリ形式でトレーニングされた重みのみを含む[ checkpoint ](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)形式のファイルのコレクションに重みを格納します。チェックポイントには、次のものが含まれます。

- 1 つ以上のモデルの重みのシャード。
- どの重みがどのシャードに格納されているかを示すインデックスファイル。

一台のマシンでモデルをトレーニングしている場合は、接尾辞が `.data-00000-of-00001` のシャードが 1 つあります。

## 手動で重みを保存する

`Model.save_weights` メソッドを使用して手動で重みを保存します。デフォルトでは、`tf.keras`、特に `save_weights` は、<code>.ckpt</code> 拡張子を持つ TensorFlow の<a>チェックポイント</a>形式を使用します ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html) に `.h5` 拡張子を付けて保存する方法については、[モデルの保存とシリアル化](https://www.tensorflow.org/guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format)ガイドを参照してください)。

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## モデル全体の保存

[`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) を呼ぶことで、モデルのアーキテクチャや重み、トレーニングの設定を単一のファイル/フォルダに保存できます。これにより、オリジナルの Python コード (*) にアクセスせずにモデルを使えるように、モデルをエクスポートできます。オプティマイザの状態も復旧されるため、中断したところからトレーニングを再開できます。

モデル全体を 2 つの異なるファイル形式 (`SavedModel`と`HDF5`) で保存できます。TensorFlow `SavedModel` 形式は、TF2.x のデフォルトのファイル形式ですが、モデルは `HDF5` 形式で保存できます。モデル全体を 2 つのファイル形式で保存する方法の詳細については、以下の説明をご覧ください。

完全に動作するモデルを保存すると TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model)、[HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) で読み込んで、ブラウザ上でトレーニングや実行したり、TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_)、[HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_)) を用いてモバイルデバイス上で実行できるよう変換することもできるので非常に便利です。

*カスタムのオブジェクト (クラスを継承したモデルやレイヤー) は保存や読み込みを行うとき、特別な注意を必要とします。以下の**カスタムオブジェクトの保存**を参照してください。 

### SavedModel フォーマットとして

SavedModel 形式は、モデルをシリアル化するもう 1 つの方法です。この形式で保存されたモデルは、`tf.keras.models.load_model` を使用して復元でき、TensorFlow Serving と互換性があります。SavedModel をサービングおよび検査する方法についての詳細は、[SavedModel ガイド](https://www.tensorflow.org/guide/saved_model)を参照してください。以下のセクションでは、モデルを保存および復元する手順を示します。

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

SavedModel 形式は、protobuf バイナリと TensorFlow チェックポイントを含むディレクトリです。保存されたモデルディレクトリを調べます。

In [ ]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

保存したモデルから新しい Keras モデルを再度読み込みます。

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

復元されたモデルは、元のモデルと同じ引数でコンパイルされます。読み込まれたモデルで評価と予測を実行してみてください。

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

### HDF5ファイルとして

Keras は [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) の標準に従ったベーシックな保存形式も提供します。 

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

保存したファイルを使ってモデルを再作成します。

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

正解率を検査します。

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

Keras は、アーキテクチャを検査することでモデルを保存します。この手法ではすべてが保存されます。

- 重みの値
- モデルのアーキテクチャ
- モデルのトレーニング構成（`.compile()` メソッドに渡すもの）
- あれば、オプティマイザとその状態（中断した所からトレーニングを再開するため）

Keras は `v1.x` (`tf.compat.v1.train` にあります) のオプティマイザを保存できません。これらはチェックポイントと互換性がないためです。v1.x のオプティマイザでは、オプティマイザの状態を読み込ませてモデルを再度コンパイルする必要があります。


### カスタムオブジェクトの保存

SavedModel 形式を使用している場合は、このセクションをスキップできます。HDF5 と SavedModel の主な違いは、HDF5 はオブジェクト構成を使用してモデルアーキテクチャを保存するのに対し、SavedModel は実行グラフを保存することです。したがって、SavedModels は、元のコードを必要とせずに、サブクラス化されたモデルやカスタムレイヤーなどのカスタムオブジェクトを保存できます。

カスタムオブジェクトを HDF5 に保存するには、次の手順を実行します。

1. オブジェクトで `get_config` メソッドを定義し、オプションで `from_config` クラスメソッドを定義します。
    - `get_config(self)` は、オブジェクトの再作成に必要なパラメータの JSON シリアル化可能なディクショナリを返します。
    - `from_config(cls, config){/code0 }は、<code data-md-type="codespan">get_config` から返された構成を使用して新しいオブジェクトを作成します。デフォルトでは、この関数は構成を初期化 kwargs (`return cls(**config)`) として使用します。
2. モデルを読み込むときに、オブジェクトを `custom_objects` 引数に渡します。引数は、文字列クラス名を Python クラスにマッピングするディクショナリである必要があります。(例: `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`）

カスタムオブジェクトと `get_config` の例については、[レイヤーとモデルを最初から作成する](https://www.tensorflow.org/guide/keras/custom_layers_and_models)チュートリアルを参照してください。
